# Centre of a satellite orbit

In [3]:
import numpy as np
import numpy.matlib as npmat
import sys
import datetime
sys.path.append('/home/ishuwa_tinda/local/src/Python/radar')
import matplotlib.pyplot as plt
from measurement.measurement import state_vector_ESAEOD

## Create a state vector object

In [5]:
orbit_file = "/home/ishuwa/local/src/Python/radar/orbit/S1B_OPER_AUX_POEORB_OPOD_20180825T110641_V20180804T225942_20180806T005942.EOF"
sv = state_vector_ESAEOD()
start_date = datetime.datetime(2018,8,4,23,0,2)
end_date = datetime.datetime(2018,8,4,23,1,2)
svecs = sv.readStateVectors(orbit_file, start_date, end_date)

In [10]:
s_vectors = sv.measurementData

In [55]:
A = np.array([sv.satEQM(x, 0.0)[3:] for x in s_vectors]).T
R = (np.array(s_vectors).T)[0:3,:]
V = (np.array(s_vectors).T)[3:,:]

In [56]:
R

array([[ -577398.508853,  -552683.859382,  -527980.819268,  -503292.991937,
         -478623.970311],
       [ 4947574.657982,  4897649.682902,  4847142.650836,  4796059.255773,
         4744405.260122],
       [-5033185.646195, -5084568.932062, -5135381.160956, -5185616.671903,
        -5235269.869399]])

In [57]:
V

array([[ 2471.925047,  2470.94462 ,  2469.603342,  2467.902239,
         2465.842385],
       [-4963.206578, -5021.694678, -5079.616823, -5136.966137,
        -5193.735812],
       [-5166.691106, -5109.870625, -5052.480793, -4994.528128,
        -4936.019205]])

In [58]:
A

array([[-0.07996622, -0.11610086, -0.1521355 , -0.18806513, -0.22388474],
       [-5.87687678, -5.8206273 , -5.76368685, -5.70606218, -5.64776011],
       [ 5.65336425,  5.71062309,  5.76723329,  5.82318882,  5.87848375]])

In [59]:
a = A.dot(np.diag(np.sqrt(1.0/np.sum(A*A, axis=0))))

In [60]:
(dummy, N) = a.shape

In [74]:
v = np.sum(a, axis=1)
r = np.sum(R, axis=1)
rv = np.sum(np.sum(R*a))
y = r - rv*v/N
M = (np.eye(3)  + np.outer(v,v)/(N**2-np.inner(v,v)))/N
RMtx = N*np.eye(3) - 1.0/N*np.outer(v,v)
print v/N
print r
print rv*v/N
print M
print RMtx
print np.linalg.inv(RMtx)
c = M.dot(y)
print c

[-0.01864259 -0.70668224  0.70712078]
[ -2639980.149751  24232831.507615 -25674022.280515]
[   656936.55954858  24902403.99675233 -24917857.48225086]
[[   0.49849188   11.3148909   -11.32191249]
 [  11.3148909   429.1120162  -429.1781827 ]
 [ -11.32191249 -429.1781827   429.64451436]]
[[ 4.99826227 -0.06587195  0.06591283]
 [-0.06587195  2.50300106  2.49854848]
 [ 0.06591283  2.49854848  2.49990101]]
[[   0.49849188   11.3148909   -11.32191249]
 [  11.3148909   429.1120162  -429.1781827 ]
 [ -11.32191249 -429.1781827   429.64451436]]
[-658394.21302266  -96419.72675395 -188750.99855614]


In [70]:
## Check the total error

In [73]:
print np.sum(R*R)
print rv*rv/N
print (y.T).dot(M.dot(y))
print np.sum(np.sum(R*R)) - rv*rv/N - (y.T).dot(M.dot(y))

2.50728431436e+14
2.48350015208e+14
2.37795773941e+12
458488678.492


In [72]:
print np.sqrt(np.sum(R*R, axis=0))

[ 7081302.26606623  7081332.71988695  7081362.53071188  7081391.705628
  7081420.25180448]


In [78]:
C = np.matlib.repmat(c,5,1).T
print R
print np.sqrt(np.sum((R-C)*(R-C), axis=0))

[[ -577398.508853  -552683.859382  -527980.819268  -503292.991937
   -478623.970311]
 [ 4947574.657982  4897649.682902  4847142.650836  4796059.255773
   4744405.260122]
 [-5033185.646195 -5084568.932062 -5135381.160956 -5185616.671903
  -5235269.869399]]
[ 6994067.96596842  6994350.37622264  6994638.41795682  6994931.98971387
  6995230.99028905]
